In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import hashlib
from datetime import datetime

In [17]:
df = pd.read_csv('../../datas/credit_card_purchases.csv')
df = df.dropna().reset_index(drop=True)

In [18]:
def to_hash(cc_num):
    return int(hashlib.sha256(cc_num.encode()).hexdigest(), 16) % 10**8

df['customer_id'] = df['cc_num'].astype('str').apply(to_hash)

In [19]:
df = df.drop(columns=['Unnamed: 0', 'cc_num', 'first', 'last', 'street', 'lat', 'long', 'merch_lat', 'merch_long', 'merch_zipcode', 'trans_num', 'zip'])

In [20]:
df['trans_date_trans_time'] = df['trans_date_trans_time'].str[:10]
df['trans_date_trans_time'] = pd.to_datetime(df['trans_date_trans_time'])
df['dob'] = pd.to_datetime(df['dob'])
df = df.rename(columns={'trans_date_trans_time': 'trans_date'})

In [21]:
def find_age(dob):
    today = datetime.today()
    age = today.year - dob.year

    if (today.month, today.day) < (dob.month, dob.day):
        age -= 1
    return age

df['age'] = df['dob'].apply(find_age)

In [22]:
df['merchant'] = df['merchant'].apply(lambda x: x.replace('fraud_', ''))

In [23]:
df.sample()

,trans_date,merchant,category,amt,gender,city,state,city_pop,job,dob,unix_time,is_fraud,customer_id,age
760712,2019-12-25,Stoltenberg-Beatty,shopping_pos,771.42,F,Rocky Mount,MO,1847,Tax inspector,1988-10-26,1356473973,1,75091458,35


In [24]:
df = df.iloc[:, [0, 12, 1, 2, 11, 3, 4, 13, 8, 9, 5, 6, 7, 10]]

In [25]:
df.sample(10)

,trans_date,customer_id,merchant,category,is_fraud,amt,gender,age,job,dob,city,state,city_pop,unix_time
180769,2019-04-18,13317096,Champlin and Sons,home,0,12.11,M,83,Sub,1941-03-30,Burbank,OK,471,1334789107
916811,2020-03-24,86968298,Sporer Inc,gas_transport,0,60.60,M,37,Hydrographic surveyor,1987-02-22,Sauk Rapids,MN,16163,1364123979
674564,2019-12-02,15584199,Windler LLC,kids_pets,0,29.96,F,42,Contractor,1981-10-24,Phoenix,AZ,1312922,1354476654
250059,2019-05-25,49712420,Berge LLC,gas_transport,0,75.96,M,39,Site engineer,1985-06-20,Sachse,TX,20328,1337917409
783322,2019-12-31,60187843,"Tillman, Dickinson and Labadie",gas_transport,0,70.62,M,63,"Development worker, community",1961-01-31,Tamaroa,IL,2135,1356954035
706533,2019-12-11,56507444,"Schoen, Nienow and Bauch",personal_care,0,44.64,F,36,Camera operator,1988-03-25,Birmingham,IA,888,1355234968
652422,2019-11-25,69989404,"Wintheiser, Dietrich and Schimmel",misc_pos,0,84.85,F,36,Geophysicist/field seismologist,1988-03-21,Munith,MI,2523,1353814740
616894,2019-11-06,59023085,Lubowitz-Walter,kids_pets,0,95.75,M,96,Insurance underwriter,1927-10-24,Pelham,NC,3402,1352228513
868377,2020-02-27,23909590,Jast and Sons,food_dining,0,14.91,F,34,"Sales professional, IT",1990-01-17,Altonah,UT,302,1361994199
896277,2020-03-14,55046758,Johns-Hoeger,entertainment,0,6.47,F,69,Trading standards officer,1955-04-03,Duncan,MS,466,1363272452


In [26]:
# drop frauds
df = df[df['is_fraud'] == 0]
df = df.drop(columns=['is_fraud'])
dg_df = df[df['merchant'] == 'Dare-Gibson']

In [53]:
dg_df[['amt', 'age', 'dob']].describe()

,amt,age,dob
count,1414.000000,1414.000000,1414
mean,54.583296,49.839463,1974-04-09 19:39:17.567185296
min,1.000000,19.000000,1926-06-26 00:00:00
25%,18.157500,37.000000,1963-12-29 00:00:00
50%,43.350000,49.000000,1974-12-28 00:00:00
75%,75.677500,60.000000,1987-08-10 12:00:00
max,452.440000,98.000000,2004-12-30 00:00:00
std,48.822250,17.108011,NaN


In [523]:
# sns.kdeplot(x=df['age'])

In [524]:
# sns.kdeplot(x=df['amt'])

In [517]:
# sns.boxplot(x='age', y='state', data=df)

In [518]:
# sns.boxplot(x='amt', y='state', data=df)

In [519]:
# def remove_outliers(group):
#     Q1_age = group['age'].quantile(0.25)
#     Q3_age = group['age'].quantile(0.75)
#     IQR_age = Q3_age - Q1_age

#     lower_bound_age = Q1_age - 1.5 * IQR_age
#     upper_bound_age = Q3_age + 1.5 * IQR_age

#     Q1_amt = group['amt'].quantile(0.25)
#     Q3_amt = group['amt'].quantile(0.75)
#     IQR_amt = Q3_amt - Q1_amt

#     lower_bound_amt = Q1_amt - 1.5 * IQR_amt
#     upper_bound_amt = Q3_amt + 1.5 * IQR_amt

#     return group[(group['age'] >= lower_bound_amt) & (group['age'] <= upper_bound_amt) &
#                  (group['amt'] >= lower_bound_age) & (group['amt'] <= upper_bound_age)]

In [520]:
# df = df.groupby('state').apply(remove_outliers).reset_index(drop=True)

In [521]:
# sns.boxplot(x='age', y='state', data=df)

In [522]:
# sns.boxplot(data=df, x='amt', y='state')